# Trabajando con Series temporales

En el mundo del Data Science es muy frecuente encontrarse en situaciones en las que debemos analizar series temporales, esto es, datos que tienen asociado siempre una marca de tiempo. Estas series suelen tener gran cantidad de datos, con lo que a veces resulta complicado trabajar con ellas.

Veamos como hacer esto en Pandas.



## Índice

- [Introducción](#Introducci%C3%B3n)
- [Cargando los datasets](#Cargando-los-datasets)
- [Series temporales](#Series-temporales-o-timeseries)
- [DataFrame](#DataFrame)
- [Remuestreo](#Remuestreo)

## Introducción

Vamos a ver como ejemplo, dos series temporales, la [oscilación ártica](http://en.wikipedia.org/wiki/Arctic_oscillation) y la [oscilación del Atlántico Norte](http://en.wikipedia.org/wiki/North_Atlantic_oscillation), obtendremos los datasets para trabajar con ellos.

Empezamos:

In [ ]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame, Panel
pd.set_option('display.max_rows',15) # Establecemos el maximo de filas a 15

Activamos los gráficos inline para este notebook

In [ ]:
%pylab inline

Vamos a empezar a cargar datos

## Cargando los datasets

Podemos obtener el primer dataset accediendo a este [enlace](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii) o utilizando el siguiente comando en Linux o en Mac:



In [ ]:
#!curl http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii >> 'monthly.ao.index.b50.current.ascii'
!wget http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii

Cargamos el dataset en un array numPy con el método `loadtxt()`

In [ ]:
ao = np.loadtxt('monthly.ao.index.b50.current.ascii')
ao

Cada línea en el fichero consiste de tres elementos: año, mes y valor:

In [ ]:
ao[0:2]

Y la forma del array:

In [ ]:
ao.shape

## Series temporales o timeseries

Vamos a convertir estos datos en series temporales, que puedan ser manipuladas de forma natural y sencilla. El primero paso que tenemos que hacer es crear el rango de fechas para nuestra serie temporal. Viendo el fichero es sencillo, vemos que el primer registro es Enero 1959 y generar tantos timestamps como registros. La frecuencia de los datos es mensual.

In [ ]:
fechas = pd.date_range('1950-01', periods=ao.shape[0], freq='M')
fechas

Aqui estamos generando un periodo mensual desde el inicio del primer registro hasta el ultimo. 

Ahora podemos empezar a crear nuestra primera serie temporal. Las fechas de las variables fecha acturaran de índice y los valores AO, pues eso, los valores.

In [ ]:
AO = Series(ao[:,2], index=fechas)
AO

Vamos a dibujar la serie

In [ ]:
AO.plot()

O una parte solo:

In [ ]:
AO['1980':'1990'].plot()

A nivel de mes

In [ ]:
AO['1980-05':'1981-03'].plot()

Como vemos el acceso a las series temporales es muy sencillo e intuitivo. Podemos acceder por índice numérico

In [ ]:
AO[120]

Por índice de fecha:

In [ ]:
AO['1960-01']

Un año entero

In [ ]:
AO['1960']

Como siempre, podemos realizar accesos condicionales

In [ ]:
AO[AO > 0]

## DataFrame

Vamos a añadir más datos, el segundo dataset. Podemos descargar el segundo dataset desde [aqui](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/norm.nao.monthly.b5001.current.ascii) o con el siguiente comando

In [ ]:
#!curl http://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/norm.nao.monthly.b5001.current.ascii >> 'norm.nao.monthly.b5001.current.ascii'
!wget http://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/norm.nao.monthly.b5001.current.ascii

Cargamos el dataset como hemos hecho en el caso del primero:

In [ ]:
nao = np.loadtxt('norm.nao.monthly.b5001.current.ascii')
fechas_nao = pd.date_range('1950-01', periods=nao.shape[0], freq='M')
NAO = Series(nao[:,2], index=fechas_nao)
NAO

El período de tiempo es identico en los dos datasets:

In [ ]:
NAO.index


Podemos crear ahora un DataFrame que contenga los datos de AO y NAO. 

In [ ]:
aonao = DataFrame({'AO' : AO, 'NAO' : NAO})

# Forma alternativa de crear el DataFrame
'''
ao_df = pd.read_table('monthly.ao.index.b50.current.ascii', sep='\s+', header=None,)
nao_df = pd.read_table('norm.nao.monthly.b5001.current.ascii', sep='\s+', header=None,)
ao_nao_df = pd.merge(ao_df, nao_df, on=[0, 1])
ao_nao_df.columns = ['Año', 'Mes', 'AO', 'NAO']
ao_nao_df.head()
'''
aonao.head()

Vamos a dibujar el DataFrame

In [ ]:
aonao.plot(subplots=True)

Podemos referenciar cada columna por el nombre

In [ ]:
aonao['NAO']

O directamente como método del DataFrame

In [ ]:
aonao.NAO

Como siempre, podemos hacer queries complejas sobre este DataFrame, como obtener todos los valores de NAO negavitos en meses positivos de AO entre dos fechas:

In [ ]:
import datetime
aonao.loc[(aonao.AO > 0) & (aonao.NAO < 0) 
        & (aonao.index > datetime.datetime(1980,1,1)) 
        & (aonao.index < datetime.datetime(1989,1,1)),
        'NAO'].plot(kind='barh')

## Remuestreo

Pandas provee un método sencillo para remuestrear los datos a una frecuencia de datos diferente. Dos parámetros para remuestrear son el periódo de tiempo al que queremos cambiar y el método a usar para hacerlo. Por defecto el método es la media. El siguiente ejemplo calcula la media anual "A".


In [ ]:
AO_mm = AO.resample("A").mean() # https://stackoverflow.com/a/17001474/526801 para ver valores de rule
AO_mm.plot(style='g--')

Con la mediana:

In [ ]:
AO_mm = AO.resample("A").median()
AO_mm.plot()


Podemos usar nuestros propios metodos para remuestreo, por ejemplo, `np.max` para cambiar el remuestreo, en este caso a 3 años (3A)

In [ ]:
AO_mm = AO.resample("3A").apply(np.max)
AO_mm.plot()

También podemos usar varias funciones para el remuestreo

In [ ]:
AO_mm = AO.resample("A").apply(['mean', np.min, np.max])
AO_mm['1900':'2020'].plot(subplots=True)
AO_mm['1900':'2020'].plot()